In [ ]:
import torch
from torchvision.models import vgg11_bn, VGG11_BN_Weights
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = vgg11_bn(weights=VGG11_BN_Weights.DEFAULT)


/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can igno

In [22]:
from torch.utils.data import Dataset
import pandas as pd
import os

transform = VGG11_BN_Weights.DEFAULT.transforms()

class PracticeDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_frame = pd.read_csv(csv_file, header= None)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels_frame)
    
    def __getitem__(self, index):
        # Get the image
        try:
            image_name = self.labels_frame.iloc[index, 0]
            img_path = os.path.join(self.img_dir, image_name)
            image = Image.open(img_path).convert('RGB')
        except:
            raise Exception("Could not find image path")

        # Get the target
        target = torch.tensor(self.labels_frame.iloc[index, 1])

        if self.transform:
            image = self.transform(image)

        return image, target



test_dataset = PracticeDataset("../data/practice-data/practice.csv", "../data/practice-data", transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10)

In [29]:
import torch.nn as nn
model.eval()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()

with torch.no_grad():
    for X, y in test_dataloader:
        pred = model(X)
        
        loss = loss_fn(pred, y)
        print(y.shape)
        pred_idx = torch.argmax(pred, dim=1)
        print(pred_idx.shape)
        num_correct = (pred_idx == y).sum().item()
        print(num_correct)


torch.Size([3])
torch.Size([3])
3
